# memstack-agent-ui React 组件

本 notebook 演示 @memstack-agent-ui/react 包的内置组件。

## 概述

提供的组件：
- **ExecutionTimeline** - 执行时间线可视化
- **AgentProvider** - Agent 上下文提供者

## 1. ExecutionTimeline 组件

In [ ]:
# ExecutionTimeline 显示 Agent 执行过程

timeline_basic = '''
import { ExecutionTimeline } from '@memstack-agent-ui/react';
import type { TimelineEvent } from '@memstack-agent-ui/core';

interface TimelineProps {
  events: TimelineEvent[];
}

function Timeline({ events }: TimelineProps) {
  return (
    <ExecutionTimeline
      timeline={events}
      onEventClick={(event) => {
        console.log('Clicked event:', event);
      }}
    />
  );
}
'''

print(timeline_basic)

## 2. TimelineEvent 类型

In [ ]:
# TimelineEvent 接口定义

timeline_event_type = '''
interface TimelineEvent {
  /** 事件唯一标识 */
  id: string;

  /** 事件类型 */
  type: EventType;

  /** 事件数据 */
  data: Record<string, unknown>;

  /** 事件时间戳（微秒） */
  eventTimeUs: number;

  /** 事件计数器 */
  eventCounter: number;

  /** 会话 ID */
  conversationId: string;
  
  /** ISO 时间戳 */
  timestamp: string;
  
  /** 事件类别 */
  category?: EventCategory;
}
'''

print(timeline_event_type)

## 3. 时间线事件类型

In [ ]:
# 不同类型的事件在时间线上有不同展示

event_types = '''
// 思考事件
const thoughtEvent: TimelineEvent = {
  id: 'evt-1',
  type: 'thought',
  data: { content: '我需要先搜索信息...' },
  eventTimeUs: 1700000000000000,
  eventCounter: 1,
  conversationId: 'conv-123',
  timestamp: '2024-01-15T10:00:00Z',
};

// 工具调用事件
const actEvent: TimelineEvent = {
  id: 'evt-2',
  type: 'act',
  data: {
    tool_name: 'search_web',
    tool_input: { query: 'Python asyncio' },
  },
  eventTimeUs: 1700000001000000,
  eventCounter: 2,
  conversationId: 'conv-123',
  timestamp: '2024-01-15T10:00:01Z',
};

// 工具结果事件
const observeEvent: TimelineEvent = {
  id: 'evt-3',
  type: 'observe',
  data: {
    tool_name: 'search_web',
    result: 'Found 10 results...',
    duration_ms: 350,
  },
  eventTimeUs: 1700000002000000,
  eventCounter: 3,
  conversationId: 'conv-123',
  timestamp: '2024-01-15T10:00:02Z',
};

// 文本开始事件
const textStartEvent: TimelineEvent = {
  id: 'evt-4',
  type: 'text_start',
  data: {},
  eventTimeUs: 1700000003000000,
  eventCounter: 4,
  conversationId: 'conv-123',
  timestamp: '2024-01-15T10:00:03Z',
};

// 完成事件
const completeEvent: TimelineEvent = {
  id: 'evt-5',
  type: 'complete',
  data: {
    result: 'Done',
    tokens: { total: 500 },
    cost: 0.01,
  },
  eventTimeUs: 1700000004000000,
  eventCounter: 5,
  conversationId: 'conv-123',
  timestamp: '2024-01-15T10:00:04Z',
};
'''

print(event_types)

## 4. AgentProvider 完整用法

In [ ]:
# AgentProvider 属性

provider_props = '''
interface AgentProviderProps {
  /** WebSocket 服务器 URL */
  wsUrl: string;

  /** 可选的认证令牌 */
  token?: string;

  /** 可选的会话 ID */
  conversationId?: string;

  /** WebSocket 客户端选项 */
  wsClientOptions?: Partial<WebSocketClientOptions>;

  /** 子组件 */
  children: ReactNode;
}
'''

print(provider_props)

## 5. 自定义时间线渲染

In [ ]:
# 自定义时间线事件渲染

custom_timeline = '''
import { ExecutionTimeline } from '@memstack-agent-ui/react';
import type { TimelineEvent } from '@memstack-agent-ui/core';

interface CustomTimelineProps {
  events: TimelineEvent[];
}

function CustomTimeline({ events }: CustomTimelineProps) {
  return (
    <ExecutionTimeline
      timeline={events}
      // 自定义事件渲染器
      renderEvent={(event) => {
        switch (event.type) {
          case 'thought':
            return (
              <div className="event thought">
                <span className="icon">💭</span>
                <span className="content">
                  {(event.data as any).content}
                </span>
              </div>
            );

          case 'act':
            return (
              <div className="event tool-call">
                <span className="icon">🔧</span>
                <span className="tool-name">
                  {(event.data as any).tool_name}
                </span>
              </div>
            );

          case 'observe':
            return (
              <div className="event tool-result">
                <span className="icon">👀</span>
                <span className="duration">
                  {(event.data as any).duration_ms}ms
                </span>
              </div>
            );

          case 'complete':
            return (
              <div className="event complete">
                <span className="icon">✅</span>
                <span>Completed</span>
              </div>
            );

          default:
            return (
              <div className="event">
                {event.type}
              </div>
            );
        }
      }}
      // 事件点击处理
      onEventClick={(event) => {
        console.log('Event details:', event);
      }}
    />
  );
}
'''

print(custom_timeline)

## 6. 组合组件示例

In [ ]:
# 组合使用多个组件和 hooks

combined_example = '''
import {
  AgentProvider,
  ExecutionTimeline,
  useAgentChat,
  useConversation,
} from '@memstack-agent-ui/react';

// 内部聊天组件
function ChatInner() {
  const { submit, isRunning, status } = useAgentChat({
    wsUrl: 'ws://localhost:8000/agent/ws',
    conversationId: 'conv-123',
  });

  const { timeline, messages, agentState } = useConversation({
    conversationId: 'conv-123',
  });

  const [input, setInput] = useState('');

  return (
    <div className="chat-layout">
      {/* 左侧：时间线 */}
      <aside className="timeline-sidebar">
        <h3>Execution Timeline</h3>
        <ExecutionTimeline
          timeline={timeline}
          onEventClick={(e) => console.log(e)}
        />
      </aside>

      {/* 中间：消息 */}
      <main className="messages">
        {messages.map((msg) => (
          <div key={msg.id} className={`message ${msg.role}`}>
            {msg.content}
          </div>
        ))}
      </main>

      {/* 底部：输入 */}
      <footer className="input-area">
        <div className="status">
          <span className={`dot ${status}`} />
          <span>{agentState}</span>
        </div>
        <input
          value={input}
          onChange={(e) => setInput(e.target.value)}
          onKeyDown={(e) => {
            if (e.key === 'Enter' && input.trim()) {
              submit(input.trim());
              setInput('');
            }
          }}
          disabled={isRunning}
        />
        <button onClick={() => submit(input)} disabled={isRunning}>
          Send
        </button>
      </footer>
    </div>
  );
}

// 主应用组件
export function AgentChatApp() {
  return (
    <AgentProvider
      wsUrl="ws://localhost:8000/agent/ws"
      conversationId="conv-123"
    >
      <ChatInner />
    </AgentProvider>
  );
}
'''

print(combined_example)

## 7. 时间线过滤

In [ ]:
# 过滤时间线事件

timeline_filtering = '''
import { ExecutionTimeline } from '@memstack-agent-ui/react';
import { isHITLEvent, isDeltaEvent } from '@memstack-agent-ui/core';
import type { TimelineEvent, EventType } from '@memstack-agent-ui/core';

interface FilteredTimelineProps {
  events: TimelineEvent[];
  showHITLEvents?: boolean;
  showDeltaEvents?: boolean;
  categories?: EventType[];
}

function FilteredTimeline({
  events,
  showHITLEvents = true,
  showDeltaEvents = false,
  categories,
}: FilteredTimelineProps) {
  // 过滤事件
  const filteredEvents = events.filter((event) => {
    // 过滤 HITL 事件
    if (!showHITLEvents && isHITLEvent(event.type)) {
      return false;
    }

    // 过滤 delta 事件
    if (!showDeltaEvents && isDeltaEvent(event.type)) {
      return false;
    }

    // 按类别过滤
    if (categories && !categories.includes(event.type)) {
      return false;
    }

    return true;
  });

  return (
    <div>
      <div className="timeline-header">
        Showing {filteredEvents.length} of {events.length} events
      </div>
      <ExecutionTimeline timeline={filteredEvents} />
    </div>
  );
}

// 使用示例
function App() {
  const { timeline } = useConversation({ conversationId: 'conv-123' });

  return (
    <FilteredTimeline
      events={timeline}
      showHITLEvents={false}  // 不显示 HITL 事件
      showDeltaEvents={false} // 不显示 delta 事件
    />
  );
}
'''

print(timeline_filtering)

## 8. 样式定制

In [ ]:
# 组件样式定制

styling = '''
/* 执行时间线样式 */
.execution-timeline {
  font-family: monospace;
  font-size: 14px;
  padding: 16px;
  background: #f5f5f5;
  border-radius: 8px;
}

.timeline-event {
  padding: 8px 12px;
  margin: 4px 0;
  border-left: 3px solid #ddd;
  background: white;
  border-radius: 4px;
}

/* 事件类型样式 */
.timeline-event.thought {
  border-left-color: #9333ea;  /* purple */
}

.timeline-event.act {
  border-left-color: #2563eb;  /* blue */
}

.timeline-event.observe {
  border-left-color: #16a34a;  /* green */
}

.timeline-event.text_start,
.timeline-event.text_delta,
.timeline-event.text_end {
  border-left-color: #ea580c;  /* orange */
}

.timeline-event.complete {
  border-left-color: #16a34a;  /* green */
  background: #f0fdf4;
}

.timeline-event.error {
  border-left-color: #dc2626;  /* red */
  background: #fef2f2;
}

.timeline-event.hitl {
  border-left-color: #f59e0b;  /* amber */
  background: #fffbeb;
}

/* 时间戳 */
.event-timestamp {
  color: #666;
  font-size: 12px;
}

/* 图标 */
.event-icon {
  margin-right: 8px;
}
'''

print(styling)

## 总结

本 notebook 演示了 @memstack-agent-ui/react 的组件：

1. **ExecutionTimeline** - 执行时间线可视化
   - 显示事件序列
   - 支持自定义渲染
   - 支持事件过滤

2. **AgentProvider** - Agent 上下文提供者
   - 管理 WebSocket 连接
   - 提供全局状态

3. **TimelineEvent** - 时间线事件类型
   - 包含 id、type、data、timestamp 等

4. **组合使用** - 将组件和 hooks 组合使用

下一步：查看 `fe-05-integration.ipynb` 了解与后端集成。